In [1]:
# get data from billboard
#import billboard
import pandas as pd
#chart = billboard.ChartData('hot-100')

In [30]:
songs = pd.read_csv("rockdata.csv")

In [31]:
titles = songs['Song Clean'].tolist()
artists = songs['ARTIST CLEAN'].tolist()
title_artist = []
for i in range(0, len(titles)):
    title_artist.append(titles[i] + " " + artists[i])

In [4]:
# cleaning song titles and artists' names
# removes words that will not be in the query
''''
def clean(name):
    cl_str = ''
    for word in name.split():
        if word not in ['featuring', 'Featuring', '&', ',', 'X', 'x', '/', 'With', 'Carter', 'Presents', '"']:
            cl_str = cl_str + word + ' '
    return cl_str

# returns a list of strings containing song titles and artists to be queried
def get_title_artist(chart):
    talist = []
    for i in range(0,100):
        title_artist = clean(chart[i].title)+' '+clean(chart[i].artist)
        talist.append(title_artist)
    return talist

In [5]:
import json
import spotipy
import spotipy.oauth2 as oauth2

def read_api_key(filepath):
    with open(filepath, 'r') as f:
        return f.read().replace('\n','')

clientID = read_api_key('ClientID.txt')
clientSecret = read_api_key('ClientSecret.txt')
username = read_api_key('username.txt')
credentials = spotipy.oauth2.SpotifyClientCredentials(client_id = clientID, client_secret = clientSecret)
sp = spotipy.Spotify(client_credentials_manager = credentials)

In [6]:
# returns url of songs as a list
def get_url(title_artist):
    sp = spotipy.Spotify(client_credentials_manager = credentials)
    url_list = []
    for song in title_artist:
        try:
            url_list.append(sp.search(q=song, type='track', limit=1)['tracks']['items'][0]['external_urls']['spotify'])
        except:
            print(song)
    return url_list

In [46]:
# returns dataframe of audio_features
def get_audio_features(urls):
    try:
        audio_features_dict = sp.audio_features(tracks=urls)
        df = pd.DataFrame.from_dict(audio_features_dict)
        df = df.drop(columns=['type', 'id', 'uri','track_href','analysis_url'])
    except:
        print("error")
    return df

In [32]:
urls = get_url(title_artist)


retrying ...3secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...3secs


In [91]:
j=0
dataframe = pd.DataFrame()
while(True):
    if j+100 < len(urls):
        df = get_audio_features(urls[j:j+100])
        dataframe = dataframe.append(df)
        j = j+100
    else:
        df = get_audio_features(urls[j:len(urls)])
        dataframe = dataframe.append(df)
        break
dataframe = dataframe.reset_index()
dataframe['artist'] = songs['ARTIST CLEAN']

In [93]:
dataframe

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist
0,0,0.425,0.681,1,-8.604,0,0.0316,0.02290,0.000219,0.0543,0.933,131.011,279333,4,.38 Special
1,1,0.418,0.775,4,-9.591,1,0.0333,0.05310,0.001250,0.0932,0.845,123.524,247267,4,.38 Special
2,2,0.567,0.559,9,-10.175,1,0.0288,0.00888,0.001310,0.0764,0.898,127.227,280636,4,.38 Special
3,3,0.659,0.505,0,-12.773,1,0.0302,0.05300,0.000000,0.0544,0.879,128.295,241227,4,.38 Special
4,4,0.646,0.426,0,-10.629,1,0.0480,0.08580,0.072700,0.0746,0.462,124.794,359507,4,10cc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2183,83,0.561,0.763,9,-7.644,1,0.0536,0.67000,0.000003,0.0829,0.625,130.868,143160,3,ZZ Top
2184,84,0.577,0.648,9,-12.183,1,0.0383,0.00422,0.033100,0.1010,0.967,150.435,183800,4,ZZ Top
2185,85,0.580,0.885,0,-5.154,0,0.0715,0.10000,0.001570,0.1460,0.788,145.405,137933,4,ZZ Top
2186,86,0.770,0.660,2,-4.441,1,0.0278,0.04800,0.138000,0.3240,0.619,100.092,230573,4,ZZ Top
